In [ ]:
import os
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from datetime import datetime

import json

In [ ]:
path = "/dados"
myFiles = os.listdir(path)

#dados completos contendo todos os dicionarios criados (total 108)
dados = pd.DataFrame(columns = 
                     ['bocal', 
                      'pressao', 
                      'quebra_jato',
                      'vel_vento',
                      'dir_vento', 
                      'horario_experimento',
                      'dia_experimento',
                      'vazao_inicial',
                      'vazao_final',
                      'vazao_total',
                      'prec'])

# ler os dados de cada arquivo na pasta
for file in myFiles:
    data = pd.read_excel(path + "/" + file, header=None, sheet_name=None)
    #dicionario com os dados descritores (um para cada aba de cada arquivo)
    descritores = {}
    #lendo cada uma das abas e colocando os descritores nos dados
    for key in data.keys():
        descritores["bocal"] = float(data[key][1][19][0:3].replace(",","."))
        descritores["pressao"] = float(data[key][1][20][0:3].replace(",","."))
        descritores["quebra_jato"] = float(data[key][1][21][0:3].replace(",","."))
        descritores["vel_vento"] = data[key][1][23]
        descritores["dir_vento"] = data[key][1][25]
        
        descritores["horario_experimento"] = data[key][1][17].hour + (data[key][1][17].minute/60)
        
        descritores["dia_experimento"] = data[key][1][22].timetuple().tm_yday
        
        descritores["vazao_inicial"] = data[key][3][20]
        descritores["vazao_final"] = data[key][3][21]
        descritores["vazao_total"] = data[key][3][22]
        
        v = []
        for i in range(0,16):
            for j in range(0,16):
                v.append(data[key][i][j])
          
        descritores["prec"] = np.array(v)
        
        dados = dados.append(descritores, ignore_index = True)

In [ ]:
media_inicial = dados[dados['bocal'] == 2.6]['vazao_inicial'].mean().round(4)
media_final = dados[dados['bocal'] == 2.6]['vazao_final'].mean().round(4)
media_total = dados[dados['bocal'] == 2.6]['vazao_total'].mean().round(4)

dados['vazao_inicial'].fillna(media_inicial, inplace=True)
dados['vazao_final'].fillna(media_final, inplace=True)
dados['vazao_total'].fillna(media_total, inplace=True)

In [ ]:
scaled_data = dados.copy()

In [ ]:
x = scaled_data.drop('prec', axis = 1)
y = scaled_data.prec

In [ ]:
X, X_validacao, Y, Y_validacao = train_test_split(x, y, test_size = 0.07)

In [ ]:
X_treinamento_70, X_teste_30, y_treinamento_70, y_teste_30 = train_test_split(X, Y, test_size = 0.3)

In [ ]:
X_treinamento_80, X_teste_20, y_treinamento_80, y_teste_20 = train_test_split(X, Y, test_size = 0.2)

In [ ]:
X_treinamento_70.reset_index(drop=True, inplace=True)
X_teste_30.reset_index(drop=True, inplace=True)
y_treinamento_70.reset_index(drop=True, inplace=True)
y_teste_30.reset_index(drop=True, inplace=True)

X_treinamento_80.reset_index(drop=True, inplace=True)
X_teste_20.reset_index(drop=True, inplace=True)
y_treinamento_80.reset_index(drop=True, inplace=True)
y_teste_20.reset_index(drop=True, inplace=True)

X_validacao.reset_index(drop=True, inplace=True)
Y_validacao.reset_index(drop=True, inplace=True)

In [ ]:
X_validacao.to_json(r'X_validacao.json', orient='columns')
Y_validacao.to_json(r'y_validacao.json')

In [ ]:
X_treinamento_70.to_json(r'X_treinamento_70.json', orient='columns')
X_teste_30      .to_json(r'X_teste_30.json', orient='columns')
y_treinamento_70.to_json(r'y_treinamento_70.json')
y_teste_30      .to_json(r'y_teste_30.json')

In [ ]:
X_treinamento_80.to_json(r'X_treinamento_80.json', orient='columns')
X_teste_20      .to_json(r'X_teste_20.json', orient='columns')
y_treinamento_80.to_json(r'y_treinamento_80.json')
y_teste_20      .to_json(r'y_teste_20.json')